In [36]:
import requests
import pandas as pd 
import json
import calendar
import hvplot.pandas
from pathlib import Path
import seaborn as sns
%matplotlib inline

series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])

In [37]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [38]:
df_2019 = df.loc[df['Year']== '2019'].copy()
df_2019['Year Month'] = df_2019['Year'] +' '+ df_2019['Month']
df_2019 = df_2019[['Year Month', 'State','Unemployment Rate']]
df_2019.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2019 December,Alabama,2.8
2019 November,Alabama,2.7
2019 October,Alabama,2.7
2019 September,Alabama,2.8
2019 August,Alabama,3.2
...,...,...
2019 May,Wyoming,3.3
2019 April,Wyoming,3.4
2019 March,Wyoming,4.0


In [39]:
df_2019.sort_index(inplace=True,ascending=False)
df_2019 = df_2019.astype({'Unemployment Rate':'float'})
df_2019.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90)

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [40]:
stats_2019_data = Path("FInCen CSV Data/FinCen SAR Stats 2019 (Months).csv")


In [41]:
csv_file = pd.read_csv(stats_2019_data, thousands=",", index_col = 'Year Month')
csv_file

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],2
2019 January,Alabama,Depository Institution,ACH,126
2019 January,Alabama,Depository Institution,Advance Fee,4
...,...,...,...,...
2019 December,Wyoming,Securities/Futures,[Total],1
2019 December,Wyoming,[Total],[Total],60
2019 December,[Total],[Total],[Total],103088


In [42]:
csv_file=csv_file.loc[csv_file["Industry"]!="[Total]" ]
csv_file=csv_file.loc[csv_file["Suspicious Activity"]!="[Total]" ]
csv_file=csv_file.loc[csv_file["State"]!="[Total]" ]
csv_file

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,1
2019 January,Alabama,Depository Institution,ACH,126
2019 January,Alabama,Depository Institution,Advance Fee,4
2019 January,Alabama,Depository Institution,Business Loan,1
...,...,...,...,...
2019 December,Wyoming,Depository Institution,Wire,6
2019 December,Wyoming,Money Services Business (MSB),Advance Fee,2
2019 December,Wyoming,Money Services Business (MSB),Other Fraud (Type),13


In [43]:
csv_file = csv_file.reset_index()

grouped_stats = csv_file[["State", "Count", "Year Month"]]

grouped_stats = grouped_stats.groupby(['Year Month', 'State']).count()
grouped_stats

Count
Year Month     State                
2019 April     Alabama            27
               Alaska             13
               American Samoa      4
               Arizona            31
               Arkansas           24
...                              ...
2019 September Virginia           32
               Washington         27
               West Virginia      12
               Wisconsin          29
               Wyoming             8

[692 rows x 1 columns]

In [44]:
df_2019 = df_2019.reset_index()
grouped_stats = grouped_stats.reset_index()
df_2019 = df_2019.sort_values("State")

combined_df = pd.concat([df_2019, grouped_stats], axis="columns", join="inner")
combined_df

,index,Year Month,State,Unemployment Rate,Year Month,State,Count
599,24,2019 December,Alabama,2.8,2019 October,Maryland,27
588,35,2019 January,Alabama,4.4,2019 October,Guam,6
589,34,2019 February,Alabama,3.8,2019 October,Hawaii,13
590,33,2019 March,Alabama,3.6,2019 October,Idaho,12
591,32,2019 April,Alabama,2.8,2019 October,Illinois,37
...,...,...,...,...,...,...,...
9,1790,2019 October,Wyoming,3.5,2019 April,District of Columbia,15
10,1789,2019 November,Wyoming,3.9,2019 April,Florida,47
11,1788,2019 December,Wyoming,4.2,2019 April,Georgia,32
6,1793,2019 July,Wyoming,3.8,2019 April,Colorado,30
